# YouTube Script Preprocessing

- v1 Problem : 한 글자 및 추가 불용어 제거 안 되는 오류
- v2 Solution : 함수 분할 및 파이프라인 설계 -> 오류 수정 실패

- EDA 필요  
1. 채널별 업로드 현황 (1일 업로드 수, 종목별 업로드 수)
2. Text 전처리 전후 len() 비교

- Questions :  
1. 특정 날짜 기준으로 텍스트 데이터 통합하여 하나의 list로 생성한 후 분석  
2. 데이터프레임 유지한 상태로 분석 후 결과(ex. 빈도 for 워드 클라우드)만 취합

In [4]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# DB 연결
import pymysql

# 실시간 주식가격 데이터
import FinanceDataReader as fdr

# 텍스트 분석
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

from konlpy.tag import Okt
from gensim import models

from wordcloud import WordCloud

# 모델 학습 및 평가
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score, precision_score, recall_score

# 기타
import datetime
from collections import Counter
import sys
import warnings
warnings.filterwarnings('ignore')

# * 함수 선언

## (1) set_data() - YouTube Data version 수정

In [5]:
# 출처 '자체제작 감성어휘사전 모듈_ver2.ipynb'

########## 변경 사항 ##########
## ['date_h'] 생성 하지 않음
## 날짜 기준 내림차순으로 정렬 ??
## 15시 이후 날짜 조정 없음
## print 추가 등


def set_data(df,com_num = 0):  # panda, numpy, datetime, FinanceDataReader, konlpy, Counter
    # 일, 시간 나누기
    df['date'] = df['date'].astype(str)
    df['date_d'] = df['date'].str[:-2]
##     df['date_h'] = df['date'].str[-2:]
    # 타입을 데이트 타입으로 만듬
    df['date_d'] = pd.to_datetime(df['date_d'])
    
## 내림차순으로 변경 안 됨 => 해결 못 함
    df = df.sort_values(by='date_d') # 일기준으로 오름차순 정렬 
    
    if com_num != 0:
    #     통합 데이터 활용시 업종코드 지정
        df = df[df['st_cd'] == com_num] # 해당 회사만 추출
        num = str(com_num).zfill(6) # 종목코드를 6자리로 만들어줌
        
    else:
        # 특정 업종만 할 때
        num = str(df['st_cd'].iloc[0]).zfill(6)
    
    # 널값 제거
    df = df.dropna()

## 날짜 조정 불필요
#     # ============== 날짜 조정
#     ## 전일 15시 ~ 금일 15시로 날짜 조정
#     after_market = ['15', '16', '17', '18', '19', '20', '21', '22', '23']
    
#     for i in range(len(df)):
#         if df['date_h'].iloc[i] in after_market:
#             df['date_d'].iloc[i] += datetime.timedelta(1)
    
    # 주말 및 공휴일 데이터
    ### Holidays
    try:
        # 서버가 열려있을 때
        db = pymysql.connect(user='root',
                             passwd='1234',
                             host='3.35.70.166',
                             db='proj',
                             charset='utf8')

        cursor = db.cursor(pymysql.cursors.DictCursor)

        sql = "select * from holidays"
        cursor.execute(sql)
        result = cursor.fetchall()
    
        # DataFrame으로 변경
        holi = pd.DataFrame(result)
        # db 닫기 --> 안하면 메모리 잡아먹음
        db.close()
    except:
        # 서버 없을 때 깃허브에서 바로 가져옴
        db_holi = 'https://raw.githubusercontent.com/chaerui7967/stock_predict_news_and_youtube/master/Sentiment_Analysis/data/holidays.csv'
        holi = pd.read_csv(db_holi)
    
    # date 컬럼을 날짜 형식으로 변경
    holi['date'] = pd.to_datetime(holi['date'])
    
    
    ### ===================주말 및 공휴일 제외
    
    ## 주말 및 공휴일만 추출
    market_closed = holi[holi['holiday']=="O"].reset_index(drop=True)
    
    ## 휴장일 List 생성
    market_closed_list = list(market_closed['date'])
    
    # 주말 및 공휴일 제외
    while len(df[df['date_d'].isin(market_closed_list)]['date_d']) !=0:
        for i in df[df['date_d'].isin(market_closed_list)]['date_d'].index:
            df['date_d'][i] += datetime.timedelta(1)
     
    
    
    # 주식 데이터 가져오기
    start_date = str(df['date_d'].iloc[0])[:10].replace('-', '')
    end_date = str(df['date_d'].iloc[-1])[:10].replace('-', '')  # 뉴스 데이터 기준 최근 값까지의 주식 데이터
    
    stock = fdr.DataReader(num, start = start_date, end = end_date)
    
    # change 컬럼 판단 기준 close에서 수수료 보다 많은 비율이 올랐으면 1, 떨어졌으면 -1, 그 사이 0
    point = stock['Close'] * stock['Change']
    spli = stock['Close'] * 0.001  # 수수료가 0.1 % 라고 할 때
    
    stock['ud'] = np.where( point > spli, 1,
                          np.where(point < spli, -1, 0))
    
    stock = stock.reset_index() # 날짜 컬럼 가져오기
    
    s_1 = stock[['Date','ud']] # 필요한 컬럼만 가져오기
    
    s_1.columns = ['date_d','ud'] # 컬럼 명 초기화
    
    # 뉴스데이터와 주식데이터 merge
    df_1 = pd.merge(df,s_1,on='date_d')  # 영향받는 컬럼 기준으로 inner 조인 
    
## 날짜순 정렬 (위에서 내림차순 정렬 안 됨)
#     df_1 = df_1.sort_values(by=['date_d'], axis=0, ascending=False)
    
## print 추가
    print('데이터 셋 구성 완료')
    
    return df_1

### set_data() TEST

In [9]:
sampro_lgchem = set_data(sampro_lgchem)

데이터 셋 구성 완료


In [ ]:
sampro_lgchem.head()

In [ ]:
sampro_lgchem.title[0]

In [ ]:
len(sampro_lgchem.text[0])

In [ ]:
sampro_lgchem.text[0]

# * 데이터 로드

(1) DB에서 CSV 파일 다운로드  
(2) DB, Django 직접 접근?

## (1) DB에서 CSV 파일 다운로드  

In [ ]:
## 출처 '데이터 csv파일로 저장.ipynb'

# com_list = {'하이닉스' : '000660',
#            '현대차' : '005380',
#            '삼성전자' : '005930',
#            'LG화학' : '051910',
#            '셀트리온' : '068270'}
# media_list = {'아시아경제_' : 'asia_',
#              '매일경제_': 'maeil_',
#              'youtube_한국경제_':'hk_',
#              'youtube_삼프로_':'sampro_',
#              'youtube_슈카_':'suka_'}

# for n,m in media_list.items():
#     for i,j in com_list.items():

#         db = pymysql.connect(
#             user='root', 
#             passwd='1234', 
#             host='3.35.70.166', 
#             db='proj', 
#             charset = 'utf8'
#         )

#         cursor = db.cursor(pymysql.cursors.DictCursor)
#         if 'youtube' in n:
#             sql = "select * from youtube_"+ m + j +" where length(date) = 10"
#         else:
#             sql = "select * from "+ m +"news_craw_" + j +" where length(date) = 10"  # date 널값 제외
#         cursor.execute(sql)

#         result = cursor.fetchall()

#         # 데이터 프레임으로 변경
#         df = pd.DataFrame(result)

#         # csv 저장
#         df.to_csv('df_'+ n + i + '.csv', index=False)

In [6]:
com_list = {'하이닉스' : '000660',
           '현대차' : '005380',
           '삼성전자' : '005930',
           'LG화학' : '051910',
           '셀트리온' : '068270'}

## media_list에서 youtube list만 가져오기
youtube_list = {'youtube_한국경제_':'hk_', 
                'youtube_삼프로_':'sampro_',
                'youtube_슈카_':'suka_'}

company_list = ['하이닉스', '현대차', '삼성전자', 'LG화학', '셀트리온']

In [7]:
## csv 데이터 로드 & 데이터프레임명 - shape 출력

for n,m in youtube_list.items():
    for i,j in com_list.items():
        if i == '하이닉스':
            globals()['{}skhynix'.format(m)] = pd.read_csv('df_'+ n + i + '.csv')
            print('{}skhynix'.format(m), globals()['{}skhynix'.format(m)].shape)
            
        elif i == '현대차':
            globals()['{}hyundai'.format(m)] = pd.read_csv('df_'+ n + i + '.csv')
            print('{}hyundai'.format(m), globals()['{}hyundai'.format(m)].shape)
            
        elif i == '삼성전자':
            globals()['{}samsung'.format(m)] = pd.read_csv('df_'+ n + i + '.csv')
            print('{}samsung'.format(m), globals()['{}samsung'.format(m)].shape)
            
        elif i == 'LG화학':
            globals()['{}lgchem'.format(m)] = pd.read_csv('df_'+ n + i + '.csv')
            print('{}lgchem'.format(m), globals()['{}lgchem'.format(m)].shape)
            
        else:
            globals()['{}celltrion'.format(m)] = pd.read_csv('df_'+ n + i + '.csv')
            print('{}celltrion'.format(m), globals()['{}celltrion'.format(m)].shape)
            print()

hk_skhynix (504, 10)
hk_hyundai (526, 10)
hk_samsung (546, 10)
hk_lgchem (500, 10)
hk_celltrion (484, 10)

sampro_skhynix (489, 10)
sampro_hyundai (526, 10)
sampro_samsung (534, 10)
sampro_lgchem (506, 10)
sampro_celltrion (460, 10)

suka_skhynix (90, 10)
suka_hyundai (403, 10)
suka_samsung (448, 10)
suka_lgchem (64, 10)
suka_celltrion (197, 10)



### 데이터프레임 열 생략

In [ ]:
# description, url 제외
# sampro_lgchem = sampro_lgchem.iloc[:, :8]

In [ ]:
# sampro_lgchem.head()

## (2) DB, Django 직접 접근?
- 데이터 엔지니어 영역(?)

---

# * 날짜별 데이터 분리

### 특정 날짜 데이터만 가지고 오기 ?? 
- '특정 날짜' & '모든 종목' 데이터 로드
- '특정 날짜' & '모든 유튜버' 데이터 로드

#### TEST 1
- LG화학 데이터만 이용해서 테스트 중  
LG 화학 - 삼프로tv - 2021.9.8일자

In [10]:
sampro_lgchem['date_d'].value_counts()

2021-09-07    9
2021-09-09    7
2020-09-17    7
2021-01-04    6
2021-08-24    6
             ..
2020-12-17    1
2020-12-18    1
2020-12-21    1
2020-12-22    1
2021-09-15    1
Name: date_d, Length: 252, dtype: int64

In [11]:
sampro_lgchem['date_d'].value_counts().describe()

count    252.000000
mean       2.007937
std        1.350792
min        1.000000
25%        1.000000
50%        2.000000
75%        2.250000
max        9.000000
Name: date_d, dtype: float64

In [ ]:
# 특정 월만 가져오기 ??
# str(sampro_lgchem['date_d'][0])[:7]

In [12]:
# 날짜 순 정렬 TEST => set_data 함수 
sampro_lgchem = sampro_lgchem.sort_values(by=['date_d'], axis=0, ascending=False)

In [13]:
sampro_lgchem

,st_n,st_cd,ch_nm,date,title,text,views,length,description,url,date_d,ud
505,LG화학,51910,삼프로TV_경제의신과함께,2021091500,[증시 염탐정] 양 시장 강보합 출발.. 대형주 움직임은? f.이베스트투자증권 염승...,염 분리 염 승환 이사님 어서오십시오 4 5 4 5 4 우리 염소한 이사님과 함께 ...,25809,1850,삼프로TV제작사및멤버사공개채용세부공고확인링크httpstlySR1n쿨썸머여름특가반값세...,https://www.youtube.com/watch?v=e91KGwZ3wFc,2021-09-15,-1
504,LG화학,51910,삼프로TV_경제의신과함께,2021091400,"[퇴근길 라이브&백브리핑] 카카오 집어삼킨 금융소비자보호법, 지금 칼 빼든 이유는?...",[음악]으 으cool으[음악]으[음악][음악]투자를 하는 사람들의 깊이 있는데 와3...,188383,9580,삼프로TV제작사및멤버사공개채용세부공고확인링크httpswwwpage2krnotice1...,https://www.youtube.com/watch?v=0nsLNwKL40U,2021-09-14,-1
503,LG화학,51910,삼프로TV_경제의신과함께,2021091400,"[글로벌마켓 브리핑] 중국 압박하는 미국정부, 정상간 통화의 기대가 실망으로 f. ...",4 실시간 티칭 창을 봤더니 우리주님이 아침에 오라고 해서 표정 동안이 어떻게 이렇...,10914,1730,삼프로TV제작사및멤버사공개채용세부공고확인링크httpswwwpage2krnotice1...,https://www.youtube.com/watch?v=wJz_JpmgTO4,2021-09-14,-1
502,LG화학,51910,삼프로TV_경제의신과함께,2021091400,"[증시 염탐정] 양 시장 강세 출발.. 들어오는 외국인, 시장 안정되나? f.이베스...",이 황구 보시고 오셨습니다 오늘도언제나처럼 우리 증지 엄 탐정 1특수 1권의 염수 ...,39754,1537,삼프로TV제작사및멤버사공개채용세부공고확인링크httpswwwpage2krnotice1...,https://www.youtube.com/watch?v=XoLozRQFmdo,2021-09-14,-1
501,LG화학,51910,삼프로TV_경제의신과함께,2021091400,"우주산업의 혁신, 이제 민간이 주도한다_글로벌 라이브 _글로벌 라이브_ 장의성, 서...",[음악][음악]으으으222[음악]글로벌 라이브 에 오신걸 환영합니다안녕하십니까 박종...,86620,6295,삼프로TV제작사및멤버사공개채용세부공고확인링크httpswwwpage2krnotice1...,https://www.youtube.com/watch?v=8L5755txAYs,2021-09-14,-1
...,...,...,...,...,...,...,...,...,...,...,...,...
4,LG화학,51910,삼프로TV_경제의신과함께,2020010600,[심층인터뷰] '전기차'를 중심으로 2020 자동차 시장 전망,이번 조합이 이번주 금요일이 줘 4열도에서 김한진 김일구 두니우리 짐 동안 프로 김...,9730,2036,프리미엄교육자세히알아보기클릭httpswwwpage2krnotice77신과함께가파헤친...,https://www.youtube.com/watch?v=lkvLhdCBWE4,2020-01-06,-1
3,LG화학,51910,삼프로TV_경제의신과함께,2019110800,"삼성의 스마트폰 전략, 변화의 이유는? _19.11.07_박종일,박제영 [퇴근길 P...",깨알 페이지 투4 안녕하십니까 11월 7일 퇴근 낄 페이지 툰 관세 장벽을 일부 허...,11245,4587,삼성의스마트폰전략변화의이유는191107박종일박제영퇴근길Page2,https://www.youtube.com/watch?v=fE3g9xbQ4Ro,2019-11-08,1
2,LG화학,51910,삼프로TV_경제의신과함께,2019110300,주식 포트폴리오 운용 방법! 경마 분석법(?) [최준철 대표의 주식 FM가이드 05-1],5 으주식 시장의 야전사령관 최준철 대표 함께하는 주식 sm 카이드 다섯번째시간이 ...,26347,3240,최준철대표의주식FM가이드051공격개시1부,https://www.youtube.com/watch?v=ibHnxFQxYlo,2019-11-04,1
1,LG화학,51910,삼프로TV_경제의신과함께,2019090300,"누가 마지막까지 투자하는가!_19.09.03_김효진,곽상준 [오늘아침 Page2]",4[음악]깊이가 나는 경제 컨텐츠 오늘 아침 페이지 투 정용진 김도완 입니다[음악]...,13446,4244,뉴스3첫번째뉴스자막오기로SKLG로정정합니다누가마지막까지투자하는가오늘아침Page220...,https://www.youtube.com/watch?v=Hz6pPEV4tT8,2019-09-03,1


In [14]:
# lg chemical [날짜]

lgc_210914_raw = sampro_lgchem[sampro_lgchem['date_d'] == '2021-09-14']
print(lgc_210914_raw.shape)
lgc_210914_raw.head()

(5, 12)


,st_n,st_cd,ch_nm,date,title,text,views,length,description,url,date_d,ud
504,LG화학,51910,삼프로TV_경제의신과함께,2021091400,"[퇴근길 라이브&백브리핑] 카카오 집어삼킨 금융소비자보호법, 지금 칼 빼든 이유는?...",[음악]으 으cool으[음악]으[음악][음악]투자를 하는 사람들의 깊이 있는데 와3...,188383,9580,삼프로TV제작사및멤버사공개채용세부공고확인링크httpswwwpage2krnotice1...,https://www.youtube.com/watch?v=0nsLNwKL40U,2021-09-14,-1
503,LG화학,51910,삼프로TV_경제의신과함께,2021091400,"[글로벌마켓 브리핑] 중국 압박하는 미국정부, 정상간 통화의 기대가 실망으로 f. ...",4 실시간 티칭 창을 봤더니 우리주님이 아침에 오라고 해서 표정 동안이 어떻게 이렇...,10914,1730,삼프로TV제작사및멤버사공개채용세부공고확인링크httpswwwpage2krnotice1...,https://www.youtube.com/watch?v=wJz_JpmgTO4,2021-09-14,-1
502,LG화학,51910,삼프로TV_경제의신과함께,2021091400,"[증시 염탐정] 양 시장 강세 출발.. 들어오는 외국인, 시장 안정되나? f.이베스...",이 황구 보시고 오셨습니다 오늘도언제나처럼 우리 증지 엄 탐정 1특수 1권의 염수 ...,39754,1537,삼프로TV제작사및멤버사공개채용세부공고확인링크httpswwwpage2krnotice1...,https://www.youtube.com/watch?v=XoLozRQFmdo,2021-09-14,-1
501,LG화학,51910,삼프로TV_경제의신과함께,2021091400,"우주산업의 혁신, 이제 민간이 주도한다_글로벌 라이브 _글로벌 라이브_ 장의성, 서...",[음악][음악]으으으222[음악]글로벌 라이브 에 오신걸 환영합니다안녕하십니까 박종...,86620,6295,삼프로TV제작사및멤버사공개채용세부공고확인링크httpswwwpage2krnotice1...,https://www.youtube.com/watch?v=8L5755txAYs,2021-09-14,-1
500,LG화학,51910,삼프로TV_경제의신과함께,2021091400,"[증시 셔터맨] 쏠림이 심한 국내 주식, 중심 잃지 않는 종목 선정이 필요하다. ...",4아침에 도착한 말씀드렸습니다만u 머리가 이상해 보인다면여러분의 패션 감각을 의심해...,21914,1573,삼프로TV제작사및멤버사공개채용세부공고확인링크httpswwwpage2krnotice1...,https://www.youtube.com/watch?v=JOK1oGTKi1U,2021-09-14,-1


#### TEST 2
- 셀트리온 - 슈카월드

In [ ]:
suka_cell = suka_celltrion.head()

In [ ]:
# suka_cell_sd = set_data(suka_cell)

# * (2) text_pre() & stop_words() 

## youtube_pre 로 합쳐서 만들기(예정)
- 소요 시간 표시 추가(예정)

In [15]:
# 출처 '자체제작 감성어휘사전 모듈_ver2.ipynb' > def giro_dic
##표시 수정 부분

def text_pre(df,num = 0):
        
## pipeline으로 이동 => 실패
    df = set_data(df, num)
#     print('데이터 셋 구성 완료')
    
    okt = Okt()
    print('okt 모듈 활성')
    
    
    # 데이터 전처리
    # =================== nlp
    # 1. 제목, 텍스트 --> 명사, 형용사 추출
    # 태그 삭제        
    # \n \t \s 삭제
## print문 추가1
    print('태그 삭제 중')
    df['text'] = df['text'].str.replace('[\n|\t|\r|\s]','')
    df['text'] = df['text'].str.replace('[^a-zA-Z0-9가-힣]','')

## print문 추가2
    print('품사 태깅 작업 중')    
    # 토큰화 및 품사 태깅
    df['Contents_input'] = df['text'].apply(lambda x: okt.pos(x, norm=True, stem=True))
    print('태그 삭제 및 품사 태깅 완료')
    
    # 명사, 형용사 추출
    for i in range(len(df['Contents_input'])):
        wordList = []
        for j in range(len(df['Contents_input'][i])):
            if df['Contents_input'][i][j][1] == 'Noun' or df['Contents_input'][i][j][1] == 'Adjective':
                wordList.append(df['Contents_input'][i][j][0])         
        df['Contents_input'][i] = wordList
    print('명사, 형용사 추출')
    
## print문 추가3
    print('text_pre 작업 완료!')
    
    return df

In [ ]:
''' # 추후 미디어별 업데이트
media = ['오늘', '년도', '이제', '요즘', '지금', '우리나라', '한번', '사실', '처럼', \
         '증시', '개장', '얘기', '이렇다', '참고', '없다', '때문', '아니다', \
         '안녕하십니까', '안녕하세요우', '안녕하다', '어서', '오십', '시오', '모습', '내용', '정도', \
         '그렇다', '어쨌든', '어떻다', '박지영', '미래에셋', '어머님', '음악']

## 함수 내 media 전역 변수 선언
#     global media
'''

In [41]:
def stop_words(df,num = 0):
    
## media 전역 변수 선언
#     global media
    
    # 2. 불용어 처리
    # 불용어 사전 로딩
##     url = 'https://raw.githubusercontent.com/chaerui7967/stock_predict_news_and_youtube/master/%EA%B0%90%EC%84%B1%EB%B6%84%EC%84%9D/data/stopwords_ver1.txt'
    url = 'https://raw.githubusercontent.com/yoonkt200/FastCampusDataset/master/korean_stopwords.txt'
    stopwords = list(pd.read_csv(url, header=None)[0])
##     media = ['매일경제','mkcokr','무단전재및재배포금지','무단전재','재배포금지']
    media = ['실시간', '오늘', '년도', '이제', '요즘', '지금', '우리나라', '한번', '사실', '처럼', \
             '증시', '개장', '얘기', '이렇다', '참고', '없다', '때문', '아니다', \
             '안녕하십니까', '안녕하세요우', '안녕하다', '어서', '오십', '시오', '모습', '내용', '정도', \
             '그렇다', '어쨌든', '어떻다', '박지영', '미래에셋', '어머님', '음악']
    print('불용어 사전 로딩')
    
    
    for i in range(len(df['Contents_input'])):
        for word in df['Contents_input'][i]:
            if word in stopwords:
                df['Contents_input'][i].remove(word)
    print('불용어 1차 처리 완료')
    
## for문 분리                
    for i in range(len(df['Contents_input'])):
        for word in df['Contents_input'][i]:
            if word in media:
                df['Contents_input'][i].remove(word) 
    print('불용어 2차 처리 완료')
    

## 단어 길이 1 이하 제거

    
    # (1) 단어길이가 1이하인 단어 제거
    for index, wordList in enumerate(df['Contents_input']):
        if len(df['Contents_input'][index]) == 0:
            continue
        for word in wordList:
            if len(word) <= 1:
                df['Contents_input'][index].remove(word)
    print('단어 길이 1이하인 단어 제거')
    

    ''' TEST 결과 안 됨
    # (2) 단어 길이가 1 초과인 단어
    for index, wordList in enumerate(df['Contents_input']):
        if len(df['Contents_input'][index]) == 0:
            continue
        wordList = [x for x in wordList if len(x) > 1]
    print('단어 길이 2이상인 단어 리스트 생성')    
    '''

## print문 추가4
    print('stop_words 작업 완료!')
    
    return df

In [ ]:
''' from 형림님
hangeul = re.compile('[^ ㄱ-ㅣ 가-힣]')                         # 정규 표현식  → 한글 추출 규칙 : 띄어쓰기(1개)를 포함한 한글
result = hangeul.sub('', globals()[corp_list[i]]['text'][j])   # 위에 설정한 hangeul 규칙을 text에 적용
okt = Okt()                                                    # 형태소 추출
nouns = okt.nouns(result)
nouns = [x for x in nouns if len(x) > 1]                       # 한 글자 키워드 제거
nouns = [x for x in nouns if x not in stopwords]               # 불용어 제거
            
corpus = " ".join(nouns)                                       # List를 String으로 변환
globals()[corp_list[i]]['except_stopwords'][j] = corpus
'''

## text_pre() & stop_words() TEST

### Test 0

In [ ]:
## test

print(lgc_210908['Contents_input'][2][:20])

for i in range(len(lgc_210908['Contents_input'])):
    
    media = ['오늘', '년도', '이제', '요즘', '지금', '우리나라', '한번', '사실', '처럼', \
             '증시', '개장', '얘기', '이렇다', '참고', '없다', '때문', '아니다', \
             '안녕하십니까', '안녕하세요우', '안녕하다', '어서', '오십', '시오', '모습', '내용', '정도', \
             '그렇다', '어쨌든', '어떻다', '박지영', '미래에셋', '어머님', '음악']

    for word in lgc_210908['Contents_input'][i]:
#             if word in stopwords:
#                 lgc_210908['Contents_input'][i].remove(word)
        if word in media:
            lgc_210908['Contents_input'][i].remove(word)
                
print(lgc_210908['Contents_input'][2][:20])

### Test 1

In [17]:
lgc_210914_tp = text_pre(lgc_210914_raw)
lgc_210914_tp.head()

데이터 셋 구성 완료
okt 모듈 활성
태그 삭제 중
품사 태깅 작업 중
태그 삭제 및 품사 태깅 완료
명사, 형용사 추출
text_pre 작업 완료!


,st_n,st_cd,ch_nm,date,title,text,views,length,description,url,date_d,ud_x,ud_y,Contents_input
0,LG화학,51910,삼프로TV_경제의신과함께,2021091400,"[퇴근길 라이브&백브리핑] 카카오 집어삼킨 금융소비자보호법, 지금 칼 빼든 이유는?...",음악으으cool으음악으음악음악투자를하는사람들의깊이있는데와32tv저녁라이브9월13일순...,188383,9580,삼프로TV제작사및멤버사공개채용세부공고확인링크httpswwwpage2krnotice1...,https://www.youtube.com/watch?v=0nsLNwKL40U,2021-09-14,-1,-1,"[음악, 음악, 투자, 사람, 깊이, 있다, 저녁, 라이브, 위염, 수관, 님, 오..."
1,LG화학,51910,삼프로TV_경제의신과함께,2021091400,"[글로벌마켓 브리핑] 중국 압박하는 미국정부, 정상간 통화의 기대가 실망으로 f. ...",4실시간티칭창을봤더니우리주님이아침에오라고해서표정동안이어떻게이렇게정확하게아시죠아니밤늦...,10914,1730,삼프로TV제작사및멤버사공개채용세부공고확인링크httpswwwpage2krnotice1...,https://www.youtube.com/watch?v=wJz_JpmgTO4,2021-09-14,-1,-1,"[실시간, 티칭창, 우리, 주님, 아침, 오라, 고해, 표정, 안이, 어떻다, 정확..."
2,LG화학,51910,삼프로TV_경제의신과함께,2021091400,"[증시 염탐정] 양 시장 강세 출발.. 들어오는 외국인, 시장 안정되나? f.이베스...",이황구보시고오셨습니다오늘도언제나처럼우리증지엄탐정1특수1권의염수만이사님다올계십니다이사...,39754,1537,삼프로TV제작사및멤버사공개채용세부공고확인링크httpswwwpage2krnotice1...,https://www.youtube.com/watch?v=XoLozRQFmdo,2021-09-14,-1,-1,"[황구, 오늘, 처럼, 우리, 증, 지엄, 탐정, 특수, 권, 염수, 만이사, 이사..."
3,LG화학,51910,삼프로TV_경제의신과함께,2021091400,"우주산업의 혁신, 이제 민간이 주도한다_글로벌 라이브 _글로벌 라이브_ 장의성, 서...",음악음악으으으222음악글로벌라이브에오신걸환영합니다안녕하십니까박종우입니다자주식시장이런...,86620,6295,삼프로TV제작사및멤버사공개채용세부공고확인링크httpswwwpage2krnotice1...,https://www.youtube.com/watch?v=8L5755txAYs,2021-09-14,-1,-1,"[음악, 음악, 음악, 글로벌, 라이브, 걸, 환영, 안녕하십니까, 박종우, 이다,..."
4,LG화학,51910,삼프로TV_경제의신과함께,2021091400,"[증시 셔터맨] 쏠림이 심한 국내 주식, 중심 잃지 않는 종목 선정이 필요하다. ...",4아침에도착한말씀드렸습니다만u머리가이상해보인다면여러분의패션감각을의심해보셔야됩니다청담...,21914,1573,삼프로TV제작사및멤버사공개채용세부공고확인링크httpswwwpage2krnotice1...,https://www.youtube.com/watch?v=JOK1oGTKi1U,2021-09-14,-1,-1,"[아침, 착하다, 머리, 이상하다, 여러분, 패션, 감각, 의심, 청담, 봉인, 책..."


In [18]:
lgc_210914 = stop_words(lgc_210914_tp)

불용어 사전 로딩
불용어 1차 처리 완료
불용어 2차 처리 완료
stop_words 작업 완료!


In [19]:
lgc_210914.head()

,st_n,st_cd,ch_nm,date,title,text,views,length,description,url,date_d,ud_x,ud_y,Contents_input
0,LG화학,51910,삼프로TV_경제의신과함께,2021091400,"[퇴근길 라이브&백브리핑] 카카오 집어삼킨 금융소비자보호법, 지금 칼 빼든 이유는?...",음악으으cool으음악으음악음악투자를하는사람들의깊이있는데와32tv저녁라이브9월13일순...,188383,9580,삼프로TV제작사및멤버사공개채용세부공고확인링크httpswwwpage2krnotice1...,https://www.youtube.com/watch?v=0nsLNwKL40U,2021-09-14,-1,-1,"[투자, 사람, 깊이, 저녁, 라이브, 위염, 수관, 님, 저녁, 뭐또, 어류, 저..."
1,LG화학,51910,삼프로TV_경제의신과함께,2021091400,"[글로벌마켓 브리핑] 중국 압박하는 미국정부, 정상간 통화의 기대가 실망으로 f. ...",4실시간티칭창을봤더니우리주님이아침에오라고해서표정동안이어떻게이렇게정확하게아시죠아니밤늦...,10914,1730,삼프로TV제작사및멤버사공개채용세부공고확인링크httpswwwpage2krnotice1...,https://www.youtube.com/watch?v=wJz_JpmgTO4,2021-09-14,-1,-1,"[티칭창, 주님, 아침, 오라, 고해, 표정, 안이, 정확하다, 좋다, 밤늦다, 게..."
2,LG화학,51910,삼프로TV_경제의신과함께,2021091400,"[증시 염탐정] 양 시장 강세 출발.. 들어오는 외국인, 시장 안정되나? f.이베스...",이황구보시고오셨습니다오늘도언제나처럼우리증지엄탐정1특수1권의염수만이사님다올계십니다이사...,39754,1537,삼프로TV제작사및멤버사공개채용세부공고확인링크httpswwwpage2krnotice1...,https://www.youtube.com/watch?v=XoLozRQFmdo,2021-09-14,-1,-1,"[황구, 처럼, 증, 지엄, 탐정, 특수, 권, 염수, 만이사, 이사, 안녕하다, ..."
3,LG화학,51910,삼프로TV_경제의신과함께,2021091400,"우주산업의 혁신, 이제 민간이 주도한다_글로벌 라이브 _글로벌 라이브_ 장의성, 서...",음악음악으으으222음악글로벌라이브에오신걸환영합니다안녕하십니까박종우입니다자주식시장이런...,86620,6295,삼프로TV제작사및멤버사공개채용세부공고확인링크httpswwwpage2krnotice1...,https://www.youtube.com/watch?v=8L5755txAYs,2021-09-14,-1,-1,"[글로벌, 라이브, 걸, 환영, 박종우, 이다, 주식시장, 분, 많다, 사면, 분,..."
4,LG화학,51910,삼프로TV_경제의신과함께,2021091400,"[증시 셔터맨] 쏠림이 심한 국내 주식, 중심 잃지 않는 종목 선정이 필요하다. ...",4아침에도착한말씀드렸습니다만u머리가이상해보인다면여러분의패션감각을의심해보셔야됩니다청담...,21914,1573,삼프로TV제작사및멤버사공개채용세부공고확인링크httpswwwpage2krnotice1...,https://www.youtube.com/watch?v=JOK1oGTKi1U,2021-09-14,-1,-1,"[아침, 착하다, 머리, 이상하다, 패션, 감각, 의심, 청담, 봉인, 책보, 청담..."


In [20]:
lgc_210914['Contents_input'][0]

['투자',
 '사람',
 '깊이',
 '저녁',
 '라이브',
 '위염',
 '수관',
 '님',
 '저녁',
 '뭐또',
 '어류',
 '저녁',
 '시장',
 '어머',
 '코스피',
 '쪽',
 '코',
 '스타',
 '웨이브',
 '코스피',
 '삼성',
 '가제',
 '역할',
 '다해',
 '요음',
 '외국인',
 '소재',
 '업체',
 '제외',
 '대부분',
 '뭐',
 '바이오',
 '반도체',
 '기쁘다',
 '고안',
 '코스닥',
 '안좋다',
 '중용',
 '주',
 '아침',
 '비',
 '메',
 '과정',
 '코스',
 '좀차익',
 '매물',
 '보고',
 '사무소',
 '날',
 '덥다',
 '화면',
 '뿌옇다',
 '목숨',
 '정액',
 '풀',
 '불투명하다',
 '의장',
 '세',
 '상징',
 '정체',
 '체',
 '정분',
 '머리',
 '아침',
 '머리',
 '요청',
 '청담동',
 '이다',
 '청정',
 '토지',
 '청담동',
 '미용실',
 '날',
 '팩',
 '보통',
 '머리',
 '요뭐',
 '양해',
 '부탁드리다',
 '자아',
 '광고',
 '등금',
 '분',
 '살짝',
 '소개',
 '증시',
 '마감',
 '상황',
 '박진희',
 '부장',
 '파',
 '테',
 '직원',
 '용수',
 '의사',
 '현재',
 '뭐',
 '대출',
 '가능',
 '굉장하다',
 '타이',
 '트',
 '그게',
 '주식시장',
 '이주',
 '영향',
 '카카오',
 '네이버',
 '관련',
 '이야기',
 '드릴',
 '잠시',
 '광고',
 '서요',
 '박진희',
 '무장',
 '채널',
 '넘버원',
 '휴대폰',
 '보시',
 '복잡하다',
 '자료',
 '판서',
 '휴대폰',
 '아프다',
 '이다',
 '좀더',
 '편하다',
 '방법',
 '와인',
 '유플러스',
 '흑우',
 '제대로',
 '리모컨',
 '채널',
 '번',
 '보시',
 '일상',
 '즐겁다',
 '

- 단어 길이 1 이하 제거 TEST

In [37]:
stop_test = lgc_210914['Contents_input'][0]

In [ ]:
# (2) 단어 길이가 1 초과인 단어
for index, wordList in enumerate(df['Contents_input']):
    if len(df['Contents_input'][index]) == 0:
        continue
    wordList = [x for x in wordList if len(x) > 1]
print('단어 길이 2이상인 단어 리스트 생성')        

In [38]:
stop_test_af = [x for x in stop_test if len(x) > 1]
stop_test_af

['투자',
 '사람',
 '깊이',
 '저녁',
 '라이브',
 '위염',
 '수관',
 '저녁',
 '뭐또',
 '어류',
 '저녁',
 '시장',
 '어머',
 '코스피',
 '스타',
 '웨이브',
 '코스피',
 '삼성',
 '가제',
 '역할',
 '다해',
 '요음',
 '외국인',
 '소재',
 '업체',
 '제외',
 '대부분',
 '바이오',
 '반도체',
 '기쁘다',
 '고안',
 '코스닥',
 '안좋다',
 '중용',
 '아침',
 '과정',
 '코스',
 '좀차익',
 '매물',
 '보고',
 '사무소',
 '덥다',
 '화면',
 '뿌옇다',
 '목숨',
 '정액',
 '불투명하다',
 '의장',
 '상징',
 '정체',
 '정분',
 '머리',
 '아침',
 '머리',
 '요청',
 '청담동',
 '이다',
 '청정',
 '토지',
 '청담동',
 '미용실',
 '보통',
 '머리',
 '요뭐',
 '양해',
 '부탁드리다',
 '자아',
 '광고',
 '등금',
 '살짝',
 '소개',
 '마감',
 '상황',
 '박진희',
 '부장',
 '직원',
 '용수',
 '의사',
 '현재',
 '대출',
 '가능',
 '굉장하다',
 '타이',
 '그게',
 '주식시장',
 '이주',
 '영향',
 '카카오',
 '네이버',
 '관련',
 '이야기',
 '드릴',
 '광고',
 '서요',
 '박진희',
 '무장',
 '채널',
 '넘버원',
 '휴대폰',
 '보시',
 '복잡하다',
 '자료',
 '판서',
 '휴대폰',
 '아프다',
 '이다',
 '좀더',
 '편하다',
 '방법',
 '와인',
 '유플러스',
 '흑우',
 '제대로',
 '리모컨',
 '채널',
 '보시',
 '일상',
 '즐겁다',
 '영상',
 '역시',
 '수리',
 '바짝',
 '얼음',
 '돌닭',
 '기름',
 '마니커',
 '닭고기',
 '마리',
 '마니커',
 '부패하다',
 '흑체',
 '닭고기',
 '마비',
 '마니커',
 '닭볶음탕',
 '

In [40]:
print('단어 제거 전 : ', len(stop_test))
print('단어 제거 후 : ', len(stop_test_af))
print('제거한 단어 수 : ', len(stop_test)-len(stop_test_af))

단어 제거 전 :  7272
단어 제거 후 :  7020
제거한 단어 수 :  252


In [22]:
def stop_words_test1(df,num = 0):
    
## media 전역 변수 선언
#     global media
    
    # 2. 불용어 처리
    # 불용어 사전 로딩
##     url = 'https://raw.githubusercontent.com/chaerui7967/stock_predict_news_and_youtube/master/%EA%B0%90%EC%84%B1%EB%B6%84%EC%84%9D/data/stopwords_ver1.txt'
    url = 'https://raw.githubusercontent.com/yoonkt200/FastCampusDataset/master/korean_stopwords.txt'
    stopwords = list(pd.read_csv(url, header=None)[0])
##     media = ['매일경제','mkcokr','무단전재및재배포금지','무단전재','재배포금지']
    media = ['실시간', '오늘', '년도', '이제', '요즘', '지금', '우리나라', '한번', '사실', '처럼', \
             '증시', '개장', '얘기', '이렇다', '참고', '없다', '때문', '아니다', \
             '안녕하십니까', '안녕하세요우', '안녕하다', '어서', '오십', '시오', '모습', '내용', '정도', \
             '그렇다', '어쨌든', '어떻다', '박지영', '미래에셋', '어머님', '음악']
    print('불용어 사전 로딩')
    
    
    for i in range(len(df['Contents_input'])):
        for word in df['Contents_input'][i]:
            if word in stopwords:
                df['Contents_input'][i].remove(word)
    print('불용어 1차 처리 완료')
    
## for문 분리                
    for i in range(len(df['Contents_input'])):
        for word in df['Contents_input'][i]:
            if word in media:
                df['Contents_input'][i].remove(word) 
    print('불용어 2차 처리 완료')
    

## 단어 길이 1 이하 제거

    
    # (1) 단어길이가 1이하인 단어 제거
    for index, wordList in enumerate(df['Contents_input']):
        if len(df['Contents_input'][index]) == 0:
            continue
        for word in wordList:
            if len(word) <= 1:
                df['Contents_input'][index].remove(word)
    print('단어 길이 1이하인 단어 제거')
    

    ''' TEST 해보기!
    # (2) 단어 길이가 1 초과인 단어
    for index, wordList in enumerate(df['Contents_input']):
        if len(df['Contents_input'][index]) == 0:
            continue
        wordList = [x for x in wordList if len(x) > 1]
    print('단어 길이 2이상인 단어 리스트 생성')    
    '''

## print문 추가4
    print('stop_words 작업 완료!')
    
    return df

In [24]:
def stop_words_test2(df,num = 0):
    
## media 전역 변수 선언
#     global media
    
    # 2. 불용어 처리
    # 불용어 사전 로딩
##     url = 'https://raw.githubusercontent.com/chaerui7967/stock_predict_news_and_youtube/master/%EA%B0%90%EC%84%B1%EB%B6%84%EC%84%9D/data/stopwords_ver1.txt'
    url = 'https://raw.githubusercontent.com/yoonkt200/FastCampusDataset/master/korean_stopwords.txt'
    stopwords = list(pd.read_csv(url, header=None)[0])
##     media = ['매일경제','mkcokr','무단전재및재배포금지','무단전재','재배포금지']
    media = ['실시간', '오늘', '년도', '이제', '요즘', '지금', '우리나라', '한번', '사실', '처럼', \
             '증시', '개장', '얘기', '이렇다', '참고', '없다', '때문', '아니다', \
             '안녕하십니까', '안녕하세요우', '안녕하다', '어서', '오십', '시오', '모습', '내용', '정도', \
             '그렇다', '어쨌든', '어떻다', '박지영', '미래에셋', '어머님', '음악']
    print('불용어 사전 로딩')
    
    
    for i in range(len(df['Contents_input'])):
        for word in df['Contents_input'][i]:
            if word in stopwords:
                df['Contents_input'][i].remove(word)
    print('불용어 1차 처리 완료')
    
## for문 분리                
    for i in range(len(df['Contents_input'])):
        for word in df['Contents_input'][i]:
            if word in media:
                df['Contents_input'][i].remove(word) 
    print('불용어 2차 처리 완료')
    

## 단어 길이 1 이하 제거

    '''
    # (1) 단어길이가 1이하인 단어 제거
    for index, wordList in enumerate(df['Contents_input']):
        if len(df['Contents_input'][index]) == 0:
            continue
        for word in wordList:
            if len(word) <= 1:
                df['Contents_input'][index].remove(word)
    print('단어 길이 1이하인 단어 제거')
    '''

    # (2) 단어 길이가 1 초과인 단어
    for index, wordList in enumerate(df['Contents_input']):
        if len(df['Contents_input'][index]) == 0:
            continue
        wordList = [x for x in wordList if len(x) > 1]
    print('단어 길이 2이상인 단어 리스트 생성')    
    

## print문 추가4
    print('stop_words 작업 완료!')
    
    return df

In [25]:
lgc_210914_test1 = stop_words_test1(lgc_210914_tp)

불용어 사전 로딩
불용어 1차 처리 완료
불용어 2차 처리 완료
단어 길이 1이하인 단어 제거
stop_words 작업 완료!


In [26]:
lgc_210914_test2 = stop_words_test2(lgc_210914_tp)

불용어 사전 로딩
불용어 1차 처리 완료
불용어 2차 처리 완료
단어 길이 2이상인 단어 리스트 생성
stop_words 작업 완료!


In [20]:
lgc_210914['Contents_input'][0]

['투자',
 '사람',
 '깊이',
 '저녁',
 '라이브',
 '위염',
 '수관',
 '님',
 '저녁',
 '뭐또',
 '어류',
 '저녁',
 '시장',
 '어머',
 '코스피',
 '쪽',
 '코',
 '스타',
 '웨이브',
 '코스피',
 '삼성',
 '가제',
 '역할',
 '다해',
 '요음',
 '외국인',
 '소재',
 '업체',
 '제외',
 '대부분',
 '뭐',
 '바이오',
 '반도체',
 '기쁘다',
 '고안',
 '코스닥',
 '안좋다',
 '중용',
 '주',
 '아침',
 '비',
 '메',
 '과정',
 '코스',
 '좀차익',
 '매물',
 '보고',
 '사무소',
 '날',
 '덥다',
 '화면',
 '뿌옇다',
 '목숨',
 '정액',
 '풀',
 '불투명하다',
 '의장',
 '세',
 '상징',
 '정체',
 '체',
 '정분',
 '머리',
 '아침',
 '머리',
 '요청',
 '청담동',
 '이다',
 '청정',
 '토지',
 '청담동',
 '미용실',
 '날',
 '팩',
 '보통',
 '머리',
 '요뭐',
 '양해',
 '부탁드리다',
 '자아',
 '광고',
 '등금',
 '분',
 '살짝',
 '소개',
 '증시',
 '마감',
 '상황',
 '박진희',
 '부장',
 '파',
 '테',
 '직원',
 '용수',
 '의사',
 '현재',
 '뭐',
 '대출',
 '가능',
 '굉장하다',
 '타이',
 '트',
 '그게',
 '주식시장',
 '이주',
 '영향',
 '카카오',
 '네이버',
 '관련',
 '이야기',
 '드릴',
 '잠시',
 '광고',
 '서요',
 '박진희',
 '무장',
 '채널',
 '넘버원',
 '휴대폰',
 '보시',
 '복잡하다',
 '자료',
 '판서',
 '휴대폰',
 '아프다',
 '이다',
 '좀더',
 '편하다',
 '방법',
 '와인',
 '유플러스',
 '흑우',
 '제대로',
 '리모컨',
 '채널',
 '번',
 '보시',
 '일상',
 '즐겁다',
 '

In [27]:
lgc_210914_test1['Contents_input'][0]

['투자',
 '사람',
 '깊이',
 '저녁',
 '라이브',
 '위염',
 '수관',
 '저녁',
 '뭐또',
 '어류',
 '저녁',
 '시장',
 '어머',
 '코스피',
 '스타',
 '웨이브',
 '코스피',
 '삼성',
 '가제',
 '역할',
 '다해',
 '요음',
 '외국인',
 '소재',
 '업체',
 '제외',
 '대부분',
 '바이오',
 '반도체',
 '기쁘다',
 '고안',
 '코스닥',
 '안좋다',
 '중용',
 '아침',
 '메',
 '과정',
 '코스',
 '좀차익',
 '매물',
 '보고',
 '사무소',
 '덥다',
 '화면',
 '뿌옇다',
 '목숨',
 '정액',
 '불투명하다',
 '의장',
 '상징',
 '정체',
 '정분',
 '머리',
 '아침',
 '머리',
 '요청',
 '청담동',
 '이다',
 '청정',
 '토지',
 '청담동',
 '미용실',
 '보통',
 '머리',
 '요뭐',
 '양해',
 '부탁드리다',
 '자아',
 '광고',
 '등금',
 '살짝',
 '소개',
 '마감',
 '상황',
 '박진희',
 '부장',
 '직원',
 '용수',
 '의사',
 '현재',
 '대출',
 '가능',
 '굉장하다',
 '타이',
 '그게',
 '주식시장',
 '이주',
 '영향',
 '카카오',
 '네이버',
 '관련',
 '이야기',
 '드릴',
 '광고',
 '서요',
 '박진희',
 '무장',
 '채널',
 '넘버원',
 '휴대폰',
 '보시',
 '복잡하다',
 '자료',
 '판서',
 '휴대폰',
 '아프다',
 '이다',
 '좀더',
 '편하다',
 '방법',
 '와인',
 '유플러스',
 '흑우',
 '제대로',
 '리모컨',
 '채널',
 '보시',
 '일상',
 '즐겁다',
 '영상',
 '역시',
 '수리',
 '바짝',
 '얼음',
 '돌닭',
 '기름',
 '마니커',
 '닭고기',
 '마리',
 '마니커',
 '부패하다',
 '흑체',
 '닭고기',
 '마비',
 '마니커',
 '닭볶음

In [28]:
lgc_210914_test2['Contents_input'][0]

['투자',
 '사람',
 '깊이',
 '저녁',
 '라이브',
 '위염',
 '수관',
 '저녁',
 '뭐또',
 '어류',
 '저녁',
 '시장',
 '어머',
 '코스피',
 '스타',
 '웨이브',
 '코스피',
 '삼성',
 '가제',
 '역할',
 '다해',
 '요음',
 '외국인',
 '소재',
 '업체',
 '제외',
 '대부분',
 '바이오',
 '반도체',
 '기쁘다',
 '고안',
 '코스닥',
 '안좋다',
 '중용',
 '아침',
 '메',
 '과정',
 '코스',
 '좀차익',
 '매물',
 '보고',
 '사무소',
 '덥다',
 '화면',
 '뿌옇다',
 '목숨',
 '정액',
 '불투명하다',
 '의장',
 '상징',
 '정체',
 '정분',
 '머리',
 '아침',
 '머리',
 '요청',
 '청담동',
 '이다',
 '청정',
 '토지',
 '청담동',
 '미용실',
 '보통',
 '머리',
 '요뭐',
 '양해',
 '부탁드리다',
 '자아',
 '광고',
 '등금',
 '살짝',
 '소개',
 '마감',
 '상황',
 '박진희',
 '부장',
 '직원',
 '용수',
 '의사',
 '현재',
 '대출',
 '가능',
 '굉장하다',
 '타이',
 '그게',
 '주식시장',
 '이주',
 '영향',
 '카카오',
 '네이버',
 '관련',
 '이야기',
 '드릴',
 '광고',
 '서요',
 '박진희',
 '무장',
 '채널',
 '넘버원',
 '휴대폰',
 '보시',
 '복잡하다',
 '자료',
 '판서',
 '휴대폰',
 '아프다',
 '이다',
 '좀더',
 '편하다',
 '방법',
 '와인',
 '유플러스',
 '흑우',
 '제대로',
 '리모컨',
 '채널',
 '보시',
 '일상',
 '즐겁다',
 '영상',
 '역시',
 '수리',
 '바짝',
 '얼음',
 '돌닭',
 '기름',
 '마니커',
 '닭고기',
 '마리',
 '마니커',
 '부패하다',
 '흑체',
 '닭고기',
 '마비',
 '마니커',
 '닭볶음

### Test 2

In [ ]:
%%time
lgc_210914_tp = text_pre(lgc_210914_raw)

In [ ]:
lgc_210914_tp.head()

In [ ]:
%%time
lgc_210914 = stop_words(lgc_210914_tp)

In [ ]:
lgc_210914.head()

In [ ]:
lgc_210914['Contents_input'][1]

# 셀트리온 슈카 210915 TEST ==> 결과가 좋지 않음

In [43]:
suka_celltrion = set_data(suka_celltrion)

데이터 셋 구성 완료


In [46]:
# 날짜 순 정렬 TEST => set_data 함수 
suka_celltrion = suka_celltrion.sort_values(by=['date_d'], axis=0, ascending=False)
suka_celltrion

,st_n,st_cd,ch_nm,date,title,text,views,length,description,url,date_d,ud
196,셀트리온,68270,슈카월드,2021091500,"쇼크의 연속, 주린이에게 가혹한 2021년 주식시장",아[음악]wr요즘에 진짜 상황이 너무 너무힘들다고 느끼는게 여러분도 그렇게느끼실 거...,247798,743,"어렵고 딱딱한 경제,시사,금융 이야기를\n쉽고 유쾌하게 풀어내는 \n경제/시사/이슈...",https://www.youtube.com/watch?v=JPs7wc4pI6I,2021-09-15,1
195,셀트리온,68270,슈카월드,2021021000,"빅히트와 네이버 전격 합작, 글로벌 플랫폼의 꿈 / 한국 게임의 혁신은 무엇인가",으 out이건 제가 좋아하는 얘기 잠깐 드릴 텐데 얼마전에 우리의 소프트웨어가연간 ...,410553,1511,0:00 ~ 12:21 과금 위주의 길을 걷는 국내 게임사들\n12:21 ~ 25:...,https://www.youtube.com/watch?v=R4yMAbCeQhU,2021-02-10,1
194,셀트리온,68270,슈카월드,2021010200,"재미로 보는 코로나 백신과 mRNA 이야기, 노벨상을 받을 수 있을까",으 out이것은 재미로만 보세요 제가 전문가가 아니기 때문에 더 정확한 것은 뭔가님...,317221,1048,"어렵고 딱딱한 경제,시사,금융 이야기를\n쉽고 유쾌하게 풀어내는 \n경제/시사/이슈...",https://www.youtube.com/watch?v=51IA8vReimY,2021-01-04,-1
193,셀트리온,68270,슈카월드,2020121600,우선주? 신형우선주? 찬바람이 불면 떠오르는 호빵같은 배당주들,으 out지난주에 가장 뜨거웠던 주식 신풍제약 얘기부터 하도록 하겠습니다왜 뜨거운 ...,229772,1002,"0:00 ~ 8:42 기계를 능멸한 인간승리, 신풍제약\n8:42 ~ 16:43 우...",https://www.youtube.com/watch?v=n2Eet799izg,2020-12-16,-1
192,셀트리온,68270,슈카월드,2020120100,"2,633 코스피 사상 최고점 달성, 코로나 3차 웨이브와 치료제 전쟁",으 out2020 년 11월 20일날 신문기사 이런게 떴어요 주가 역대 최고 기록2...,215739,1165,"어렵고 딱딱한 경제,시사,금융 이야기를\n쉽고 유쾌하게 풀어내는 \n경제/시사/이슈...",https://www.youtube.com/watch?v=gS_e5VgV_tQ,2020-12-01,1
...,...,...,...,...,...,...,...,...,...,...,...,...
4,셀트리온,68270,슈카월드,2018011600,식빵+슈카 아재토크 6회 공무원이 선망받는 직업이 된 이유,야 정하여 반갑습니다축하해 줘 같아요 아니 2월 자 여기 있으니까 거기 앉게 됬다 ...,16569,8561,"40대 아재 2명이 바라보는 세상의 모든 이야기들,\n어렵고 딱딱한 이야기를 최대한...",https://www.youtube.com/watch?v=KNlURTebh3w,2018-01-16,-1
3,셀트리온,68270,슈카월드,2018011600,[아재토크 하이라이트] 거침없이 가즈아~!! 상하이지수,저 여러분 중국 주식 이에요 야 이 더러운 프라이즈 1993년에104.9 룩 이었어...,15171,297,식빵과 슈카의 아재토크 17회에서 편집한 영상입니다. \n17회 풀버전을 보실 분은...,https://www.youtube.com/watch?v=x7ITkivGc9k,2018-01-16,-1
2,셀트리온,68270,슈카월드,2018011500,"[풀버전] 식빵과 슈카의 아재토크 35회 (4) - 큰 강을 둘러싼 분쟁, 메콩강과...",거 같으면 아 어 지 깬 아스트라 솔로 면서 아직 새끼가 파닭 반론바람에 내가 눕게...,6896,2400,"40대 아재 2명이 바라보는 세상의 모든 이야기들,\n어렵고 딱딱한 이야기를 최대한...",https://www.youtube.com/watch?v=naOc1vdltHs,2018-01-15,1
1,셀트리온,68270,슈카월드,2018011500,[풀버전] 식빵과 슈카의 아재토크 35회 (5) - 아마존강의 커다란 물고기,진짠가하겠어 그래서 이렇게 야 이거 해 누가 정말 한 크기의 뱀이 있어요 이거누가 ...,4538,1220,"40대 아재 2명이 바라보는 세상의 모든 이야기들,\n어렵고 딱딱한 이야기를 최대한...",https://www.youtube.com/watch?v=fySowd6wbk4,2018-01-15,1


In [47]:
suka_celltrion.date_d.value_counts()

2018-02-01    6
2018-02-05    6
2018-01-25    5
2018-03-12    4
2018-01-26    4
             ..
2020-06-19    1
2020-06-22    1
2020-06-23    1
2020-06-25    1
2019-05-22    1
Name: date_d, Length: 145, dtype: int64

In [45]:
cell_210915_raw = suka_celltrion[suka_celltrion['date_d'] == '2021-09-15']
print(cell_210915_raw.shape)
cell_210915_raw.head()

(1, 12)


,st_n,st_cd,ch_nm,date,title,text,views,length,description,url,date_d,ud
196,셀트리온,68270,슈카월드,2021091500,"쇼크의 연속, 주린이에게 가혹한 2021년 주식시장",아[음악]wr요즘에 진짜 상황이 너무 너무힘들다고 느끼는게 여러분도 그렇게느끼실 거...,247798,743,"어렵고 딱딱한 경제,시사,금융 이야기를\n쉽고 유쾌하게 풀어내는 \n경제/시사/이슈...",https://www.youtube.com/watch?v=JPs7wc4pI6I,2021-09-15,1


In [48]:
cell_210915 = text_pre(cell_210915_raw)
cell_210915 = stop_words(cell_210915)

데이터 셋 구성 완료
okt 모듈 활성
태그 삭제 중
품사 태깅 작업 중
태그 삭제 및 품사 태깅 완료
명사, 형용사 추출
text_pre 작업 완료!
불용어 사전 로딩
불용어 1차 처리 완료
불용어 2차 처리 완료
단어 길이 1이하인 단어 제거
stop_words 작업 완료!


In [49]:
cell_210915_list = []

for i in cell_210915.Contents_input:
    for j in i:
        cell_210915_list.append(j)
        
count = Counter(cell_210915_list)
words = dict(count.most_common())

words_df = pd.DataFrame.from_dict([words]).T.reset_index()
words_df.columns = ['words', 'freq']

words_df.to_csv('cell_210915_wc.csv', header=True, index=False, encoding = 'utf-8-sig')

In [51]:
words_df.head(50)

,words,freq
0,올해,12
1,정말,8
2,대비,7
3,생각,7
4,있다,7
5,강남,7
6,좋다,6
7,뭐,6
8,주식,5
9,진짜,4


# 삼성전자 한경 210913 TEST

In [ ]:
# hk_skhynix (504, 10)
# hk_hyundai (526, 10)
# hk_samsung (546, 10)
# hk_lgchem (500, 10)
# hk_celltrion (484, 10)

# sampro_skhynix (489, 10)
# sampro_hyundai (526, 10)
# sampro_samsung (534, 10)
# sampro_lgchem (506, 10)
# sampro_celltrion (460, 10)

# suka_skhynix (90, 10)
# suka_hyundai (403, 10)
# suka_samsung (448, 10)
# suka_lgchem (64, 10)
# suka_celltrion (197, 10)

In [52]:
hk_samsung = set_data(hk_samsung)

데이터 셋 구성 완료


In [53]:
# 날짜 순 정렬 TEST => set_data 함수 
hk_samsung = hk_samsung.sort_values(by=['date_d'], axis=0, ascending=False)
hk_samsung

,st_n,st_cd,ch_nm,date,title,text,views,length,description,url,date_d,ud
508,삼성전자,5930,한국경제TV,2021091600,삼성전자·SK하이닉스 비중확대 해야 할 이유(노근창)/ 주식경제 이슈분석 / 한국경제TV,4 오랄 3 십일조를 판 외국인의지금 반도체를 향해서는 이달 들어서조금씩 매수를 해...,1428,941,○ 노근창 (현대차증권 리서치센터장)\n○ 곽수종 앵커\n○ 손현정 앵커\n\n\n...,https://www.youtube.com/watch?v=WRdlEVeBsCs,2021-09-16,-1
507,삼성전자,5930,한국경제TV,2021091600,장기로 자산을 늘리기 위해 000은 반드시 투자하세요(김대종)/ 주식경제 이슈분석 ...,[음악]월가에서는 9월 자가 계속 조심해야한다 고 경고를 보내고 있습니다미국의 렐리...,770,979,○ 김대종 (세종대 경영학과 교수)\n○ 곽수종 앵커\n○ 손현정 앵커\n\n\n#...,https://www.youtube.com/watch?v=XrDghL0zhv0,2021-09-16,-1
506,삼성전자,5930,한국경제TV,2021091600,"전기차 배터리 소재 실리콘 음극재 도약, 4분기가 기다려지는 이 종목 / 윤혁진 S...",시장의 보는 눈을 키우는 시간입니다오늘은 sk 증권 20 개 팀장이 신윤 혁진 연구...,1633,425,"[ 나의투자 ] \n▶ 출연자 \n김두언,이혜수 / 앵커\n윤혁진 / SK증권 연구...",https://www.youtube.com/watch?v=ufZDplEHbr8,2021-09-16,-1
505,삼성전자,5930,한국경제TV,2021091500,美 주식 ‘공갈빵’ vs 中·日 주식 ‘진실빵’ 논쟁(Ⅱ) 韓 주식은 어느 쪽이 될...,4 작년에 연인들 사이에서 부산의이바구 길 투어가 되면서 이바구공작소 에서 빠는 진...,4534,2073,○한상춘 한국경제TV 논설위원 \n\n△대한민국의 경제를 보세요!\n△빠르고 정확한...,https://www.youtube.com/watch?v=paYO75Gk2HU,2021-09-15,1
504,삼성전자,5930,한국경제TV,2021091500,파운드리 반도체 삼파전(삼성·인텔·TSMC)…승자는?(최양오)/ 주식경제 이슈분석 ...,으[음악]이번에 국내 시장에 대한 심층적인이야기 나눠보겠습니다 아이팩 테크플랫폼 규...,7934,1071,○ 최양오 (현대경제연구원 고문)\n○ 곽수종 앵커\n○ 손현정 앵커\n\n\n#최...,https://www.youtube.com/watch?v=LWMOpaYjEh0,2021-09-15,1
...,...,...,...,...,...,...,...,...,...,...,...,...
5,삼성전자,5930,한국경제TV,2012100500,삼성전자 3분기 `깜짝 실적`..IT 부품주 관심 [2012.10.05 방송],wewe별장 개장하고 이제 17분 가까워지고 있습니다 2000선 회복했지만 다시20...,9,538,한국경제TV 공식웹사이트 ☞ http://www.wowtv.co.kr\n\n대한민국...,https://www.youtube.com/watch?v=o0tL52ik7YQ,2012-10-05,1
3,삼성전자,5930,한국경제TV,2012092700,"이재용 삼성전자 사장, 스마트폰 직접 챙긴다 [2012.09.26 방송]",갤럭시노트 2 출시와 함께 이재용 삼성전자 사장이 미국으로 출국했습니다4 이재용 사...,465,102,한국경제TV 공식웹사이트 ☞ http://www.wowtv.co.kr\n\n대한민국...,https://www.youtube.com/watch?v=xVxv1gab6AQ,2012-09-27,1
2,삼성전자,5930,한국경제TV,2012091300,"삼성전자, 반도체 중국생산 시대 연다",삼성전자가 중국 산시성 시안 시에서 낸드플래시 반도체 공장 기공식을열었습니다삼성전자...,164,124,한국경제TV 공식웹사이트 ☞ http://www.wowtv.co.kr\n\n대한민국...,https://www.youtube.com/watch?v=ypLbsc_H5kc,2012-09-13,1
1,삼성전자,5930,한국경제TV,2012073000,삼성전자 또 사상최대 실적 2분기 영업이익 6조7천억원,소식입니다 삼성전자가 또다시 사상 최대 실적을 경신했습니다 갤럭시 시리즈의 위력을 ...,36,145,한국경제TV 공식웹사이트 ☞ http://www.wowtv.co.kr\n\n대한민국...,https://www.youtube.com/watch?v=aoiSui8kuio,2012-07-30,1


In [54]:
hk_samsung.date_d.value_counts()

2021-09-07    12
2021-09-08    11
2021-09-13    10
2021-09-09    10
2021-09-10     8
              ..
2020-04-06     1
2020-04-07     1
2020-04-14     1
2020-04-21     1
2012-06-26     1
Name: date_d, Length: 345, dtype: int64

In [55]:
ss_210913_raw = hk_samsung[hk_samsung['date_d'] == '2021-09-13']
print(ss_210913_raw.shape)
ss_210913_raw.head()

(10, 12)


,st_n,st_cd,ch_nm,date,title,text,views,length,description,url,date_d,ud
495,삼성전자,5930,한국경제TV,2021091100,"추석 이후 주목할 종목 심텍 · 삼화전기 / 심텍, 삼화전기 / 이경준 유안타증권 ...",4 끝장 전략 시간입니다 매주 금 일날 만날 수 있는 분이죠 유 않다증권 압구정 봄...,5893,567,º 성공투자 오후증시\nMC 나민호 / 이경은\n출연 이경준 유안타증권 압구정본부 ...,https://www.youtube.com/watch?v=RCfCdbCvcGk,2021-09-13,1
490,삼성전자,5930,한국경제TV,2021091100,"플랫폼주 조정, 매수 기회! / 오기석 크래프트테크놀로지스 법인장 / 커넥트 오늘내...",커넥트 오늘과 내일 시간입니다 매주금 일 난 만나는 부인 조 크래프터테크놀로지 스의...,2148,386,º 성공투자 오후증시\nMC 나민호 / 이경은\n출연 오기석 크래프트테크놀로지스 법...,https://www.youtube.com/watch?v=YPD9zRXb6go,2021-09-13,1
491,삼성전자,5930,한국경제TV,2021091100,조여오는 유동성 축소와 플랫폼 기업 규제 속 전략은?(김대종)/ 주식경제 이슈분석 ...,[음악]4 우리 시장 9월 시작이 녹록치 는않았는데요 현재 이 코스피 에서도카카오와...,2243,583,○ 김대종 (세종대 경영학과 교수)\n○ 곽수종 앵커\n○ 손현정 앵커\n\n\n#...,https://www.youtube.com/watch?v=JKqFzBLzUuI,2021-09-13,1
492,삼성전자,5930,한국경제TV,2021091100,"공정위도 나선 빅테크 네이버카카오 규제 우려, 반사이익 기업은/이재만 하나금융투자 ...",4 시장의 보는 눈을 키우는시간입니다 오늘은 여의도에서 별명이붙어 주신 분입니다파수...,5226,651,"[ 나의투자 ] \n▶ 출연자 \n김두언,이혜수 / 앵커\n이재만 / 하나금융투자 ...",https://www.youtube.com/watch?v=qg5T86hv7i8,2021-09-13,1
494,삼성전자,5930,한국경제TV,2021091100,"[경제전쟁 꾼 27] 약해지는 테이퍼링 우려, 유동성 파티 이어질까? (삼프로TV ...",고용 이슈가 1 그야말로 쇼코수중으로 놀랐죠엄청 많습니다이슈 하나만으로는 스테이크 ...,201571,3081,꾼의 시선으로 경제를 본다!\n- 경제와 시사를 아우르는 프로중의 프로 김동환 소장...,https://www.youtube.com/watch?v=xdazq1qrlFc,2021-09-13,1


In [56]:
ss_210913 = text_pre(ss_210913_raw)
ss_210913 = stop_words(ss_210913)

데이터 셋 구성 완료
okt 모듈 활성
태그 삭제 중
품사 태깅 작업 중
태그 삭제 및 품사 태깅 완료
명사, 형용사 추출
text_pre 작업 완료!
불용어 사전 로딩
불용어 1차 처리 완료
불용어 2차 처리 완료
단어 길이 1이하인 단어 제거
stop_words 작업 완료!


In [67]:
ss_210913_list = []

for i in ss_210913.Contents_input:
    for j in i:
        ss_210913_list.append(j)
        
count = Counter(ss_210913_list)
words = dict(count.most_common())

words_df = pd.DataFrame.from_dict([words]).T.reset_index()
words_df.columns = ['words', 'freq']

words_df.to_csv('ss_210913_wc.csv', header=True, index=False, encoding = 'utf-8-sig')

In [58]:
words_df.head(50)

,words,freq
0,생각,98
1,부분,68
2,미국,64
3,좋다,61
4,시장,57
5,관련,47
6,경우,47
7,말씀,45
8,종목,45
9,중국,42


# 셀트리온 한경 210909 TEST

In [ ]:
# hk_skhynix (504, 10)
# hk_hyundai (526, 10)
# hk_samsung (546, 10)
# hk_lgchem (500, 10)
# hk_celltrion (484, 10)

# sampro_skhynix (489, 10)
# sampro_hyundai (526, 10)
# sampro_samsung (534, 10)
# sampro_lgchem (506, 10)
# sampro_celltrion (460, 10)

# suka_skhynix (90, 10)
# suka_hyundai (403, 10)
# suka_samsung (448, 10)
# suka_lgchem (64, 10)
# suka_celltrion (197, 10)

In [59]:
hk_celltrion = set_data(hk_celltrion)

데이터 셋 구성 완료


In [60]:
# 날짜 순 정렬 TEST => set_data 함수 
hk_celltrion = hk_celltrion.sort_values(by=['date_d'], axis=0, ascending=False)
hk_celltrion

,st_n,st_cd,ch_nm,date,title,text,views,length,description,url,date_d,ud
458,셀트리온,68270,한국경제TV,2021091400,중대한 시장위험 임박하지 않았다!(김한진)/ 주식경제 이슈분석 / 한국경제TV,[음악]이제 우리 주식시장의 저명한이코노미스트 에 또 고견을 들어보는시간입니다오늘 ...,7427,1211,○ 김한진 (KTB투자증권 수석연구위원)\n○ 곽수종 앵커\n○ 손현정 앵커\n\n...,https://www.youtube.com/watch?v=2r_mxd3gzt0,2021-09-14,1
457,셀트리온,68270,한국경제TV,2021091200,"운임은 최고가 인데 해운 말고 조선주(+기자재)를? / 현대중공업, HMM, 삼성중...",[음악]소의 공급과 수요의 서서 에 불균형때문에 그런데요이정기 선사 컨테이너 선사들...,27554,1010,MC : 이진우 GFM투자연구소장\n 권다영 캐스터\n\n출연 : 민...,https://www.youtube.com/watch?v=vy8412FKVwo,2021-09-13,-1
456,셀트리온,68270,한국경제TV,2021091100,"카카오,네이버 수익률 부진하지만 계속 순매수중(전윤호)/ 주식경제 주식 초고수의 선...",주식초보 세상의 1% 포트폴리오엿보는 시간입니다 미래에셋 증권디지털 솔루션 팀 전유...,4639,692,○ 전윤호 (미래에셋증권 디지털솔루션팀 수석)\n○ 곽수종 앵커\n○ 손현정 앵커\...,https://www.youtube.com/watch?v=It5XuV3li7s,2021-09-13,-1
455,셀트리온,68270,한국경제TV,2021091100,"[경제전쟁 꾼 27] 약해지는 테이퍼링 우려, 유동성 파티 이어질까? (삼프로TV ...",고용 이슈가 1 그야말로 쇼코수중으로 놀랐죠엄청 많습니다이슈 하나만으로는 스테이크 ...,199419,3081,꾼의 시선으로 경제를 본다!\n- 경제와 시사를 아우르는 프로중의 프로 김동환 소장...,https://www.youtube.com/watch?v=xdazq1qrlFc,2021-09-13,-1
454,셀트리온,68270,한국경제TV,2021091000,"세방전지 자회사 효과 기대 / 세방전지, 와이엠텍 / 윤주호 엄브렐라리서치 대표 /...",미스터 스몰캡 입니다 스몰캡 전문가를모시고 투자 아이디어를 듣는시간입니다 오늘은 엄...,1961,517,º 성공투자 오후증시\nMC 나민호 / 이경은\n출연 윤주호 엄브렐라리서치 대표 \...,https://www.youtube.com/watch?v=Y2w64nTg5lA,2021-09-10,-1
...,...,...,...,...,...,...,...,...,...,...,...,...
7,셀트리온,68270,한국경제TV,2013042300,시장의 주요 지지선 위협 다음 주 투자전략은_2013년 4월 19일 방송,아 네 이번주 정리를 해 봤구요 다음주 투자 전략을 세워 보도록하겠습니다오늘이 시간...,3,823,[한국경제TV - www.wowtv.co.kr] - 대한민국을 넘어 세계 증권경제 ...,https://www.youtube.com/watch?v=rm2jmakEQ08,2013-04-23,1
2,셀트리온,68270,한국경제TV,2013041800,"셀트리온, 공매도 논란 `가열`_2013년 4월 17일",쌀 툴이 온 공매도 논란이 연일 주식 시장을 혼란스럽게 하고 있습니다오늘은 셀트리온...,474,135,[한국경제TV - www.wowtv.co.kr] - 대한민국을 넘어 세계 증권경제 ...,https://www.youtube.com/watch?v=8cFE1vX1MKI,2013-04-18,-1
1,셀트리온,68270,한국경제TV,2013041800,[종목진단] 이재칠 셀트리온 소액주주동호회 회장_2013년 4월 17일,예 이궁 매도 논란으로 어이 뜨거운 종목이 이쪽 5 싸게 시가총액 1위인셀트리온 입...,1974,314,[한국경제TV - www.wowtv.co.kr] - 대한민국을 넘어 세계 증권경제 ...,https://www.youtube.com/watch?v=f17eCIY9As8,2013-04-18,-1
3,셀트리온,68270,한국경제TV,2013041800,"셀트리온, 공매도 논란 `가열`_2013년 4월 17일",날씨 촌 여러분 안녕하십니까 대한민국 대표 경제 유수 한국경제 좋습니다쌀 트리 온 ...,968,141,[한국경제TV - www.wowtv.co.kr] - 대한민국을 넘어 세계 증권경제 ...,https://www.youtube.com/watch?v=oG8zUIcHa6M,2013-04-18,-1


In [61]:
hk_celltrion.date_d.value_counts()

2014-01-09    5
2021-02-01    5
2021-07-06    5
2020-12-03    4
2021-09-09    4
             ..
2020-07-23    1
2020-07-20    1
2020-07-16    1
2020-07-14    1
2013-02-13    1
Name: date_d, Length: 345, dtype: int64

In [62]:
cell_210909_raw = hk_celltrion[hk_celltrion['date_d'] == '2021-09-09']
print(cell_210909_raw.shape)
cell_210909_raw.head()

(4, 12)


,st_n,st_cd,ch_nm,date,title,text,views,length,description,url,date_d,ud
453,셀트리온,68270,한국경제TV,2021090900,"바이오리더스 / 플랫폼 기반 신약 개발, 자궁경부전암 치료제 / 김형철 와우넷 파...",더 오늘 종목은 바이오리더스 라는종목을 좀 가지고 왔습니다 즉 국제꿀벌이 네 맛있게...,1715,277,[바이오리더스]\n플랫폼 기반 신약 개발…가치 재평가\n자궁경부전암 치료제 연내 3...,https://www.youtube.com/watch?v=ryHepjsgqOo,2021-09-09,-1
452,셀트리온,68270,한국경제TV,2021090900,"4분기 투자 유망 건설주 찾아보니 / GS건설, 태영건설, 금호건설, 와이엔텍 / ...",4 엘리트 뷰 시간입니다 최근들어시장은 굉장히 어렵습니다 그러나반등을 시작하는 업종...,3963,386,º 성공투자 오후증시\nMC 나민호 / 이경은\n출연 신서정 SK증권 연구위원\n\...,https://www.youtube.com/watch?v=wzi8lMUG_30,2021-09-09,-1
451,셀트리온,68270,한국경제TV,2021090900,당분간 뚜렷한 상승은 없다. 네이버카카오 정부 규제영향과 주도산업/허재환 유진투자증...,4 시장의 본을 눈을 키우는시간입니다 오늘은 여의도 브레인 허재와 뉴 지쳐 증권 이...,23674,695,"[ 나의투자 ] \n▶ 출연자 \n김두언,이혜수 / 앵커\n허재환 / 유진투자증권 ...",https://www.youtube.com/watch?v=GDFvjD0_kyg,2021-09-09,-1
450,셀트리온,68270,한국경제TV,2021090900,"삼화전기 / 전해콘덴서 업체, 실적 대비 저평가 / 김지욱 와우넷 파트너 / 증시하...",삼화전기 입니다 사과 저 예 삼아전기를 가져왔고 예 어떤 삼아 저같은 경우에는 알루...,822,109,[삼화전기]\n- 알루미늄 전해콘덴서 업체…실적 대비 저평가\n- 5G·자동차 전장...,https://www.youtube.com/watch?v=CWRNBLRP_hk,2021-09-09,-1


In [63]:
cell_210909 = text_pre(cell_210909_raw)
cell_210909 = stop_words(cell_210909)

데이터 셋 구성 완료
okt 모듈 활성
태그 삭제 중
품사 태깅 작업 중
태그 삭제 및 품사 태깅 완료
명사, 형용사 추출
text_pre 작업 완료!
불용어 사전 로딩
불용어 1차 처리 완료
불용어 2차 처리 완료
단어 길이 1이하인 단어 제거
stop_words 작업 완료!


In [66]:
cell_210909_list = []

for i in cell_210909.Contents_input:
    for j in i:
        cell_210909_list.append(j)
        
count = Counter(cell_210909_list)
words = dict(count.most_common())

words_df = pd.DataFrame.from_dict([words]).T.reset_index()
words_df.columns = ['words', 'freq']

words_df.to_csv('cell_210909_wc.csv', header=True, index=False, encoding = 'utf-8-sig')

In [65]:
words_df.head(50)

,words,freq
0,부분,25
1,좋다,18
2,규제,13
3,종목,12
4,말씀,12
5,기업,10
6,같다,10
7,건설,10
8,최근,9
9,성장,9


# SK하이닉스 삼프로 2109?? TEST

In [ ]:
# hk_skhynix (504, 10)
# hk_hyundai (526, 10)
# hk_samsung (546, 10)
# hk_lgchem (500, 10)
# hk_celltrion (484, 10)

# sampro_skhynix (489, 10)
# sampro_hyundai (526, 10)
# sampro_samsung (534, 10)
# sampro_lgchem (506, 10)
# sampro_celltrion (460, 10)

# suka_skhynix (90, 10)
# suka_hyundai (403, 10)
# suka_samsung (448, 10)
# suka_lgchem (64, 10)
# suka_celltrion (197, 10)

In [71]:
sampro_skhynix = set_data(sampro_skhynix)

데이터 셋 구성 완료


In [72]:
# 날짜 순 정렬 TEST => set_data 함수 
sampro_skhynix = sampro_skhynix.sort_values(by=['date_d'], axis=0, ascending=False)
sampro_skhynix

,st_n,st_cd,ch_nm,date,title,text,views,length,description,url,date_d,ud
487,하이닉스,660,삼프로TV_경제의신과함께,2021091500,"[증시 셔터맨] 골목상권 사업 축소하는 카카오, 주가 영향은? f. 한국투자증권 박...",4 i 뭐 제가 워낙에 멋진지 많은분들 어디 임용 일이냐고 물어 보시는분들이 많은데...,30884,1371,삼프로tv가만드는주린이탈출콘텐츠에출연할구독자를모집합니다httpsebcpage2krn...,https://www.youtube.com/watch?v=o4z3A6lgNPk,2021-09-15,-1
486,하이닉스,660,삼프로TV_경제의신과함께,2021091500,"성장주·가치주 구분 없이 이익이 좋을 기업들을 찾아보면?_박병창, 유명간, 염승환",으으으으으으아으깊이가 다른 경제 컨텐츠 웨어 34tb 온다 10 라이브 적용 시에김...,285425,7204,삼프로tv가만드는주린이탈출콘텐츠에출연할구독자를모집합니다httpsebcpage2krn...,https://www.youtube.com/watch?v=9ojkt9d3qpU,2021-09-15,-1
485,하이닉스,660,삼프로TV_경제의신과함께,2021091500,[심층 인터뷰] 실적 Peak-out 걱정 없는 기업들은 어떤 회사들? f.미래에셋...,유명한 영국의 님 어서 오세요안녕하세요 i 우리집 로딩을 처음에어 장 읽어 줘 예 ...,24693,1959,삼프로tv가만드는주린이탈출콘텐츠에출연할구독자를모집합니다httpsebcpage2krn...,https://www.youtube.com/watch?v=mU6-ieH-xXQ,2021-09-15,-1
484,하이닉스,660,삼프로TV_경제의신과함께,2021091500,"[뉴스3] 올해 31조 팔아치운 외국인, 이달엔 반도체주 줍줍?",예 에뉴스 삶처럼 시작하겠습니다 그쵸 먼저첫번째 뉴스는 외국인들 뭐 지난8월에 등으...,9301,1216,삼프로tv가만드는주린이탈출콘텐츠에출연할구독자를모집합니다httpsebcpage2krn...,https://www.youtube.com/watch?v=xE1X95ibSk8,2021-09-15,-1
483,하이닉스,660,삼프로TV_경제의신과함께,2021091500,[증시 염탐정] 양 시장 강보합 출발.. 대형주 움직임은? f.이베스트투자증권 염승...,염 분리 염 승환 이사님 어서오십시오 4 5 4 5 4 우리 염소한 이사님과 함께 ...,25320,1850,삼프로TV제작사및멤버사공개채용세부공고확인링크httpstlySR1n쿨썸머여름특가반값세...,https://www.youtube.com/watch?v=e91KGwZ3wFc,2021-09-15,-1
...,...,...,...,...,...,...,...,...,...,...,...,...
4,하이닉스,660,삼프로TV_경제의신과함께,2019092000,"전술적 자산배분의 기본 원칙_19.09.19_영주 닐슨, 박제영 [퇴근길 Page2]",4 오늘 하루도 수고하셨습니다3% 티비 저녁 라이브 퇴근길 페이지 투의 진행을 맡은...,13934,3759,LIVE퇴근길Page220190919구독과좋아요는사랑입니다평일매일아침7시30분매일저...,https://www.youtube.com/watch?v=x9nBDF5NtjA,2019-09-20,-1
3,하이닉스,660,삼프로TV_경제의신과함께,2019091900,"손정의가 하이닉스를 살릴까?_19.09.18_김영우, 박제영_[퇴근길 Page2]",[음악]오늘 하루도 수고하셨습니다 3% tv 전용 나이 9퇴근 깰 페이지 투[음악]...,17635,4069,손정의가하이닉스를살릴까190918김영우박제영퇴근길Page2구독과좋아요는사랑입니다평일...,https://www.youtube.com/watch?v=xjCtRQV18jE,2019-09-19,1
2,하이닉스,660,삼프로TV_경제의신과함께,2019072400,[퇴근길 Page2] 돈이 돈을 버는 순간까지 00처럼 아껴라_19.07.23 /...,tou으[음악]아 미치겠네 지금 팔아야 되는데 이 놈의 오줌이 꼭 결정적인 순간에 ...,11635,3563,퇴근길Page2돈이돈을버는순간까지00처럼아껴라190723김유라박제영,https://www.youtube.com/watch?v=9mV9by6Mg-4,2019-07-24,-1
1,하이닉스,660,삼프로TV_경제의신과함께,2019072300,[퇴근길 Page2] 가치 투자자 박성진 대표가 말하는 전업 투자자의 길_19.07...,later 으 으[음악]빚을 내서 투자를 해야 할까 아니면 다 팔고 차입을 챙기는 ...,29107,4793,퇴근길Page2가치투자자박성진대표가말하는전업투자자의길190722박성진박제영,https://www.youtube.com/watch?v=IhqE2bkqW5I,2019-07-23,1


In [73]:
sampro_skhynix.date_d.value_counts()

2021-08-17    8
2021-08-18    7
2021-08-19    7
2021-09-13    6
2021-09-09    6
             ..
2020-12-29    1
2020-12-24    1
2020-12-22    1
2020-12-18    1
2019-07-01    1
Name: date_d, Length: 262, dtype: int64

In [74]:
skhynix_210913_raw = sampro_skhynix[sampro_skhynix['date_d'] == '2021-09-13']
print(skhynix_210913_raw.shape)
skhynix_210913_raw.head()

(6, 12)


,st_n,st_cd,ch_nm,date,title,text,views,length,description,url,date_d,ud
481,하이닉스,660,삼프로TV_경제의신과함께,2021091200,연준의 정책과 국내 주식시장 전망 [신과대화 : 김한진 KTB투자증권 수석이코노미스트 ],ntr 못해요 뭘로 우리의 ohpower222좀 그 유혹적인 있을것 아예 하지만뭐 ...,81258,5515,삼프로TV제작사및멤버사공개채용세부공고확인링크httpswwwpage2krnotice1...,https://www.youtube.com/watch?v=DZ1KZcIMF24,2021-09-13,1
480,하이닉스,660,삼프로TV_경제의신과함께,2021091100,"쉬어가는 미국증시, 고민되는 포트폴리오_글로벌 라이브 _글로벌 라이브_ 강구현, 이...",[음악][음악]으으[음악]으2빨리 3% tv 글로벌 라이브 오신여러분을 환영합니다 ...,104231,6245,삼프로TV제작사및멤버사공개채용세부공고확인링크httpswwwpage2krnotice1...,https://www.youtube.com/watch?v=_Cz7TBuxNPI,2021-09-13,1
479,하이닉스,660,삼프로TV_경제의신과함께,2021091100,[심층 인터뷰] 카카오만의 문제냐? 카카오 먼저의 문제냐? f. 이베스트투자증권 윤...,회원을 금요일 6월 10일아저희가 장이 어렵고 헷갈리고 어떻게해야될지 잘 모를 때 ...,92416,3227,삼프로TV제작사및멤버사공개채용세부공고확인링크httpswwwpage2krnotice1...,https://www.youtube.com/watch?v=wD76Wmz95p0,2021-09-13,1
478,하이닉스,660,삼프로TV_경제의신과함께,2021091100,"[글로벌마켓 브리핑] 미-중 정상 간 두번째 통화, 증시에 어떤 영향이? f. 미래...",괴로움 안녕하세요 댓글을 보다보니까제 머리 모양이 왜 이러냐고 글쎄요저도 몰랐는데어...,20309,1718,삼프로TV제작사및멤버사공개채용세부공고확인링크httpswwwpage2krnotice1...,https://www.youtube.com/watch?v=7rYF8YQabLU,2021-09-13,1
477,하이닉스,660,삼프로TV_경제의신과함께,2021091100,"[퇴근길라이브&백브리핑] 윤지호 센터장이 말하는 강한 반등의 조건 / ISA통장, ...",[음악]으 으a[음악]cole 음 으[음악]으으[음악][음악]피는 대화 3 4tb ...,204080,9530,삼프로TV제작사및멤버사공개채용세부공고확인링크httpswwwpage2krnotice1...,https://www.youtube.com/watch?v=x1TCrEyR-J8,2021-09-13,1


In [75]:
skhynix_210913 = text_pre(skhynix_210913_raw)
skhynix_210913 = stop_words(skhynix_210913)

데이터 셋 구성 완료
okt 모듈 활성
태그 삭제 중
품사 태깅 작업 중
태그 삭제 및 품사 태깅 완료
명사, 형용사 추출
text_pre 작업 완료!
불용어 사전 로딩
불용어 1차 처리 완료
불용어 2차 처리 완료
단어 길이 1이하인 단어 제거
stop_words 작업 완료!


In [76]:
skhynix_210913_list = []

for i in skhynix_210913.Contents_input:
    for j in i:
        skhynix_210913_list.append(j)
        
count = Counter(skhynix_210913_list)
words = dict(count.most_common())

words_df = pd.DataFrame.from_dict([words]).T.reset_index()
words_df.columns = ['words', 'freq']

words_df.to_csv('skhynix_210913_wc.csv', header=True, index=False, encoding = 'utf-8-sig')

In [77]:
words_df.head(50)

,words,freq
0,좋다,289
1,시장,195
2,생각,170
3,중국,168
4,미국,154
5,말씀,152
6,굉장하다,126
7,많다,108
8,종목,101
9,상황,96


# * 파이프라인(보류)

In [ ]:
# df_clean = youtube_pre(df)
# df_clean.head()

In [ ]:
# ex. lgchem_210809_list

com_date_list = []

for i in com_date.Contents_input:
    for j in i:
        com_date_list.append(j)
        
count = Counter(com_date_list)
words = dict(count.most_common())

words_df = pd.DataFrame.from_dict([words]).T.reset_index()
words_df.columns = ['words', 'freq']

words_df.to_csv('com_date_wc.csv', header=True, index=False, encoding = 'utf-8-sig')

# 목표 : STT 텍스트 정제

## FastText

https://inahjeon.github.io/fasttext/

In [29]:
from gensim.models import FastText

In [ ]:
# gunzip cc.ko.300.vec.gz
# gunzip cc.ko.300.bin.gz

In [30]:
ko_model = models.fasttext.load_facebook_model('cc.ko.300.bin')

In [ ]:
for w, sim in m_fasttext.similar_by_word('파이썬', 10):
    print(f'{w}: {sim}')

In [ ]:
model = FastText(data, size=100, window=5, min_count=5, workers=4, sg=1)

In [ ]:
model.wv.most_similar('')

https://vhrehfdl.tistory.com/77

In [32]:
# embedding = FastText(lgc_210914['Contents_input'][0], size=100, window=7, negative=3, min_count=5)
embedding = FastText(lgc_210914['Contents_input'][0], window=7, negative=3, min_count=5)
embedding.save('fasttext.model')

In [33]:
model = FastText.load('fasttext.model')

In [34]:
# print(model.wv['티칭창'])

[-2.5865317e-03 -3.8800368e-04  3.5524275e-03 -2.1607096e-03
 -2.3786856e-03  7.1477302e-04  1.7091107e-03  1.2328442e-03
 -2.7699741e-03  2.4295569e-04 -1.7320713e-03  2.0132323e-03
  3.4837208e-03 -7.4854051e-04 -2.6100241e-03  8.2819071e-04
 -6.3785445e-04  5.2880780e-03 -1.3630780e-03  2.4474559e-03
  2.0023251e-03 -3.0574251e-03  3.5545893e-03  1.6764858e-03
  2.2446816e-03  2.6308221e-03 -3.2320537e-04 -3.8382588e-03
  3.2423770e-03 -1.9063704e-03 -3.8454544e-03  1.0989885e-03
 -2.7893891e-03 -1.6144030e-03  5.0837075e-04  1.1414988e-03
  2.7172960e-04 -3.5560690e-03 -1.5779581e-03 -8.5963117e-04
 -1.6861191e-03 -1.6496499e-03 -1.8934374e-03 -3.4711186e-03
  2.1728599e-03 -3.0230763e-04 -1.9615588e-03 -1.6256232e-03
  2.5016570e-03  1.1014943e-03 -1.4571085e-03 -9.2227594e-04
  1.2993051e-04 -2.8032620e-04  1.3599679e-03  2.4908383e-03
 -5.8836554e-04 -7.1876422e-03  3.4908373e-03  3.9292860e-04
 -1.4511369e-03  4.3322914e-03 -2.3795899e-03 -1.3235770e-04
 -1.0918585e-03 -4.70315

In [36]:
print(model.wv.most_similar('티칭창'))

[('힘', 0.2857452929019928), ('레', 0.268752783536911), ('음', 0.25929611921310425), ('혁', 0.24410496652126312), ('계', 0.2408999502658844), ('츠', 0.23554031550884247), ('순', 0.23359213769435883), ('인', 0.2310313731431961), ('향', 0.22450362145900726), ('응', 0.2218310832977295)]


## Py-Hansell